## Sagemaker

In [2]:
# !pip install tensorflow

In [3]:
# !pip install keras

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
import tensorflow
import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, layers, optimizers
from keras.layers import Dropout

import os
import gc
from timeit import default_timer as timer

Using TensorFlow backend.


In [2]:
import os
import sagemaker
import numpy as np
from sagemaker.tensorflow import TensorFlow
from tensorflow.python.keras.preprocessing.image import load_img

No handlers could be found for logger "sagemaker"


In [53]:
sagemaker_session = sagemaker.Session()

In [54]:
role = sagemaker.get_execution_role()

In [55]:
bucket = "sagemaker-all-cnn"
key = "Data"
key_output = "output"                   # Path from the bucket's root to the dataset
train_instance_type='ml.m4.xlarge'      # The type of EC2 instance which will be used for training
deploy_instance_type='ml.m4.xlarge'     # The type of EC2 instance which will be used for deployment
hyperparameters={
    "learning_rate": 0.001,
    "decay": 0.0001
}

In [56]:
train_input_path = "s3://{}/{}/training/".format(bucket, key)
validation_input_path = "s3://{}/{}/validation/".format(bucket, key)

In [57]:
estimator = TensorFlow(
  entry_point=os.path.join(os.path.dirname('__file__'), "Model_Scripts/Model_3x3C1D_drop25.py"),             # Your entry script
  role=role,
  framework_version="1.12.0",               # TensorFlow's version
  hyperparameters=hyperparameters,
  training_steps=100,
  evaluation_steps=30,
  train_instance_count=1,                   # "The number of GPUs instances to use"
  train_instance_type=train_instance_type,
)

In [58]:
print("Training ...")
estimator.fit({'training': train_input_path, 'eval': validation_input_path})

Training ...
2020-09-21 16:24:04 Starting - Starting the training job...
2020-09-21 16:24:06 Starting - Launching requested ML instances......
2020-09-21 16:25:29 Starting - Preparing the instances for training.........
2020-09-21 16:26:36 Downloading - Downloading input data......
2020-09-21 16:28:02 Training - Training image download completed. Training in progress..2020-09-21 16:28:02,785 INFO - root - running container entrypoint
2020-09-21 16:28:02,785 INFO - root - starting train task
2020-09-21 16:28:02,803 INFO - container_support.training - Training starting
2020-09-21 16:28:06,726 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2020-09-21 16:28:06,726 INFO - tf_container - {"environment": "cloud", "cluster": {"master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
2020-09-21 16:28:06,726 INFO - tf_container - ---------------------------------------------------------
2020-09-21 16:28:06,726 INFO - tf_container - creating RunConfig:
2